In [1]:
import httpx
from pysolr import Solr

CREATE CORE

In [2]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'authorities',
        'configSet': '_default',
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores', 
    headers=headers, 
    json=json_data, 
    timeout=None)
response

<Response [200 OK]>

In [3]:
headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'create': {
        'name': 'acervo',
        'configSet': '_default',
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores', 
    headers=headers, 
    json=json_data, 
    timeout=None)
response

<Response [200 OK]>

Authorities Schema

In [7]:
url = "http://localhost:8983/api/cores/authorities/schema"

headers = {
    'Content-type': 'application/json',
}

CREATE FIELD

In [4]:
headers = {
    'Content-type': 'application/json',
}

json_data = {
    'add-field': {
        'name': 'general_search',
        'type': 'text_general',
        'stored': False,
    },
}

response = httpx.post(
    'http://localhost:8983/api/cores/authorities/schema', 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

Add a New Copy Field Rule

In [6]:
json_data = {
    'add-copy-field': {
        'source': 'fullerName',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200]>

In [8]:
json_data = {
    'add-copy-field': {
        'source': 'name',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

In [9]:
json_data = {
    'add-copy-field': {
        'source': 'variants',
        'dest': [
            'general_search',
        ],
    },
}

response = httpx.post(
    url, 
    headers=headers, 
    json=json_data,
    timeout=None)

response

<Response [200 OK]>

In [12]:
url = "http://localhost:8983/solr/admin/cores?action=REINDEXCOLLECTION&name=authorities"
r = httpx.post(url)
r

json_data = {
    'action': {
        'name': 'authorities',
        'configSet': '_default',
    },
}
r = "http://localhost:8983/solr/admin/collections?action=REINDEXCOLLECTION&name=techproducts_v2"


<Response [400 Bad Request]>

In [2]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":5,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-2"}},\n  "status":"OK"}\n'

In [3]:
r = solr.search(q="*:*")

In [5]:
r.docs

[{'id': 'sh85103704',
  'type': ['Topic'],
  'authority': ['Poesia'],
  'variants': ['Poesia - Filosofia', 'Poemas', 'Versos (Poesia)'],
  'broaders': ['Literatura'],
  'narrowers': ['Estrofes',
   'poesia natalina',
   'poesia infantil',
   'Triveni',
   'Poesia macarrônica',
   'poesia religiosa',
   'Poesia dos lugares',
   'Poesia política',
   'Poesia de dialeto',
   'poesia de rua',
   'poesia visual',
   'Poesia biográfica',
   'Poesia patriótica',
   'centavos',
   'hinos',
   'Verso livre',
   'Poesia elegíaca',
   'poesia imagista',
   'poesia especulativa',
   'Poesia encontrada',
   'poesia experimental',
   'Poesia, Moderna',
   'baladas',
   'Escaldões e poesia escaldante',
   'verso autógrafo',
   'Poesia da Nova Era',
   'Abecedários',
   'poesia de hipertexto',
   'verso alexandrino',
   'poesia de fadas',
   'verso ocasional',
   'poesia pastoral',
   'lanternas',
   'poesia narrativa',
   'nazms',
   'Sestinas',
   'Sestetos',
   'poesia dadaísta',
   'Poesia anti-gu

In [4]:
solr.delete(q='*:*')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">207</int>\n</lst>\n</response>\n'

In [5]:
work = {
        "id": "wk-bk-23-4",
        "hasInstance": {"add": [f"in-bk-23-5"]}, 
    }

solr.add([work], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":110}}\n'